In [1]:
from os import listdir
from os.path import isfile, join
from datetime import datetime 
import pandas as pd
import Config
from pymongo import MongoClient
client = MongoClient(Config.DB_Hostname, Config.DB_Port)

def push_into_sim(underlying, df):
    
    #underlying = NIFTY / BANKNIFTY, dtype = Guide / Simulated
    dates = df.date.unique()
    for d in dates:
        temp = df[df.date == d]
        print(client.Strategy[f"{underlying}_Simulated_Parameters"].delete_many({"date":d}).deleted_count,f" documents deleted for {d}")
        print(len(client.Strategy[f"{underlying}_Simulated_Parameters"].insert_many(temp.to_dict('records')).inserted_ids),f" documents entered for {d}\n")

path="/Users/praneetshaw/Desktop/Strategy/CFiles/"
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
onlyfiles = [s for s in onlyfiles if ".csv" in s]

dates=[ datetime.strptime(o[:10],"%Y-%m-%d") for o in onlyfiles]
onlyfiles= [o for d,o in sorted(zip(dates,onlyfiles))]

df=pd.DataFrame()
for f in onlyfiles:
    df=pd.concat([df,pd.read_csv(f)])

df.sort_values(['date','strategy_variant'],inplace=True)
df.fillna(0, inplace=True)

In [2]:
push_into_sim("NIFTY", df)
#push_into_sim("BANKNIFTY", df)

0  documents deleted for 2024-01-18
384  documents entered for 2024-01-18

